# Homework 5: Adversarial Attacks and Defenses

Duke University

ECE661 Fall 2022

## Setup

You shouldn't have to change anything in these cells

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random

# Custom
import models
import attacks

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

device: cuda


In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size = 64, shuffle=True, )
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./data', train=False, download=True, transform=transforms.ToTensor()),
    batch_size = 64, shuffle=False, )

In [4]:
def test_model(mdl, loader, device):
    mdl.eval()
    running_correct = 0.
    running_loss = 0.
    running_total = 0.
    with torch.no_grad():
        for batch_idx,(data,labels) in enumerate(loader):
            data = data.to(device); labels = labels.to(device)
            clean_outputs = mdl(data)
            clean_loss = F.cross_entropy(clean_outputs, labels)
            _,clean_preds = clean_outputs.max(1)
            running_correct += clean_preds.eq(labels).sum().item()
            running_loss += clean_loss.item()
            running_total += labels.size(0)
    clean_acc = running_correct/running_total
    clean_loss = running_loss/len(loader)
    mdl.train()
    return clean_acc,clean_loss

## Model training

Train a model and save the checkpoint. This cell is used in Lab-1 and Lab-3

In [ ]:
## Pick a model architecture
which_net = 'B'
test_acc_arr_lab2a = []

if which_net == 'A':
    net = models.NetA().to(device)
    ## Checkpoint name for this model
    model_checkpoint = "netA_standard.pt"
if which_net == 'B':
    net = models.NetB().to(device)
    model_checkpoint = "netB_standard.pt"

## Basic training params
num_epochs = 20
initial_lr = 0.001
lr_decay_epoch = 15

optimizer = torch.optim.Adam(net.parameters(), lr=initial_lr)

## Training Loop
for epoch in range(num_epochs):
    net.train()
    train_correct = 0.
    train_loss = 0.
    train_total = 0.
    for batch_idx,(data,labels) in enumerate(train_loader):
        data = data.to(device); labels = labels.to(device)
        
        # Forward pass
        outputs = net(data)
        net.zero_grad()
        optimizer.zero_grad()
        # Compute loss, gradients, and update params
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        # Update stats
        _,preds = outputs.max(1)
        train_correct += preds.eq(labels).sum().item()
        train_loss += loss.item()
        train_total += labels.size(0)
        
    # End of training epoch
    test_acc,test_loss = test_model(net,test_loader,device)
    test_acc_arr_lab2a.append(test_acc)
    print("Epoch: [ {} / {} ]; TrainAcc: {:.5f}; TrainLoss: {:.5f}; TestAcc: {:.5f}; TestLoss: {:.5f}".format(
        epoch, num_epochs, train_correct/train_total, train_loss/len(train_loader),
        test_acc, test_loss,
    ))
    # Save model    
    torch.save(net.state_dict(), model_checkpoint)
    
    # Update LR
    if epoch == lr_decay_epoch:
        for param_group in optimizer.param_groups:
            param_group['lr'] = initial_lr*0.1

print("Done!")

In [ ]:
fig, ax = plt.subplots(1, 1)
xx = range(num_epochs)
ax.plot(xx, test_acc_arr_lab2a, label='final test accuracy: %g' % (test_acc_arr_lab2a[-1]))
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('lab1a net%s Accuracy vs Epochs' % which_net)
ax.legend()

# plt.savefig('Figures/lab1a_net%s.pdf' % which_net, dpi=500, bbox_inches='tight')

**Visualize some perturbed samples**

In [ ]:
classes = ["t-shirt", "trouser","pullover","dress","coat","sandal","shirt","sneaker","bag","boot"]
net = models.NetA().to(device)
net.load_state_dict(torch.load("netA_standard.pt"))
lab1_part = 'c2'  # possible values: 'b', 'c1', 'c2', 'd'; change this to plot lab1 b/c/d
lab1_save_name = {'b': 'PGD_attack', 'c1': 'FGSM_attack', 'c2': 'rFGSM_attack', 'd': 'FGM_L2_attack'}
EPS_list_lab1 = np.array([0, 0.005, 0.02, 0.05, 0.075, 0.1, 0.15, 0.2])
if lab1_part == 'd':
    EPS_list_lab1 = np.array([0, 0.3, 1, 1.5, 2, 3, 3.5, 4])
print('EPS_list_lab1', EPS_list_lab1)
for epsilon in EPS_list_lab1:
    for data, labels in test_loader:
        data, labels = data.to(device), labels.to(device)

        ###
        # Compute and apply adversarial perturbation to data
        # EPS in [0.0, 0.2]
        EPS = epsilon
        if lab1_part == 'b' or lab1_part == 'c1' or lab1_part == 'c2':
            assert EPS <= 0.2 and EPS >= 0.0 
        elif lab1_part == 'd':
            assert EPS <= 4 and EPS >= 0.0 
        else:
            raise KeyError("check lab1_part param")
        ITS = 10
        ALP = 1.85 * (EPS/ITS)
        if lab1_part == 'b':
            adv_data, _ = attacks.PGD_attack(model=net, device=device, dat=data, lbl=labels, eps=EPS, alpha=ALP, iters=ITS, rand_start=True)
        if lab1_part == 'c1':
            adv_data, _ = attacks.FGSM_attack(model=net, device=device, dat=data, lbl=labels, eps=EPS)
        if lab1_part == 'c2':
            adv_data, _ = attacks.rFGSM_attack(model=net, device=device, dat=data, lbl=labels, eps=EPS)
        if lab1_part == 'd':
            adv_data, _ = attacks.FGM_L2_attack(model=net, device=device, dat=data, lbl=labels, eps=EPS)
        ###
        
        # Compute preds
        with torch.no_grad():
            clean_outputs = net(data)
            _,clean_preds = clean_outputs.max(1)
            clean_preds = clean_preds.cpu().squeeze().numpy()
            adv_outputs = net(adv_data)
            _,adv_preds = adv_outputs.max(1)
            adv_preds = adv_preds.cpu().squeeze().numpy()

        # Plot some samples
        inds = random.sample(list(range(data.size(0))),6)
        plt.figure(figsize=(15,5))
        for jj in range(6):
            plt.subplot(2, 6, jj+1)
            plt.imshow(data[inds[jj],0].cpu().numpy(),cmap='gray')
            plt.axis("off")
            plt.title("clean. pred={}".format(classes[clean_preds[inds[jj]]]))
        for jj in range(6):
            plt.subplot(2, 6, 6+jj+1)
            plt.imshow(adv_data[inds[jj],0].cpu().numpy(),cmap='gray')
            plt.axis("off")
            plt.title("adv. pred={}".format(classes[adv_preds[inds[jj]]]))
        plt.suptitle("eps=%g" % EPS)
        plt.tight_layout()
        # plt.show()
        plt.savefig('Figures/lab1%s_netA_eps_%g_%s.pdf' % (lab1_part, EPS, lab1_save_name[lab1_part]), dpi=500, bbox_inches='tight')
        break

## Test Attacks - Whitebox & Blackbox

Don't forget to plot accuracy vs. epsilon curves!

In [9]:
def plot_lab2_white_black(epsilon_arr, white_acc_d, black_acc_d, label_names):
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    for label_name in label_names:
        ax[0].plot(epsilon_arr, white_acc_d[label_name], label=label_name)
        ax[0].set_xlabel('eps')
        ax[0].set_ylabel('Accuracy')
        ax[0].set_title('Whitebox Attack')

        ax[1].plot(epsilon_arr, black_acc_d[label_name], label=label_name)
        ax[1].set_xlabel('eps')
        ax[1].set_ylabel('Accuracy')
        ax[1].set_title('Blackbox Attack')
    ax[0].legend()
    ax[1].legend()
    fig.tight_layout()
    plt.savefig('Figures/lab2bcd_attacks.pdf', dpi=500, bbox_inches='tight')
    return fig, ax


def lab2_bcd_return_adv_data(model, device, dat, lbl, eps, alpha, iters, rand_start, question_label):
    if question_label == 'Random':
        return attacks.random_noise_attack(model=None, device=device, dat=dat, eps=eps)[0]
    elif question_label == 'FGSM':
        return attacks.FGSM_attack(model, device, dat, lbl, eps)[0]
    elif question_label == 'rFGSM':
        return attacks.rFGSM_attack(model, device, dat, lbl, eps)[0]
    elif question_label == 'PGD':
        return attacks.PGD_attack(model, device, dat, lbl, eps, alpha, iters, rand_start)[0]
    else:
        raise KeyError

In [ ]:
EPS_list_lab2 = np.linspace(0, 0.1, 11)
print('EPS_list_lab2', EPS_list_lab2)
lab2_label = ['Random', 'FGSM', 'rFGSM', 'PGD']
# lab2_label = {'FGSM'}
white_acc_dict, black_acc_dict = {'Random': [], 'FGSM': [], 'rFGSM': [], 'PGD': []}, {'Random': [], 'FGSM': [], 'rFGSM': [], 'PGD': []}

for epsilon in EPS_list_lab2:
    print('epsilon', epsilon)
    for q_label in lab2_label:
        print('q_label', q_label)
        white_acc_lst, black_acc_lst = [], []
        ## Load pretrained models
        whitebox = models.NetA()
        blackbox = models.NetB()

        whitebox.load_state_dict(torch.load("netA_standard.pt")) # TODO
        blackbox.load_state_dict(torch.load("netB_standard.pt")) # TODO

        whitebox, blackbox = whitebox.to(device), blackbox.to(device) 
        whitebox.eval()
        blackbox.eval()

        test_acc,_ = test_model(whitebox, test_loader, device)
        print("Initial Accuracy of Whitebox Model: ",test_acc)
        test_acc,_ = test_model(blackbox, test_loader, device)
        print("Initial Accuracy of Blackbox Model: ", test_acc)

        ## Test the models against an adversarial attack

        # TODO: Set attack parameters here
        ATK_EPS = epsilon
        ATK_ITERS = 10
        ATK_ALPHA = 1.85 * (ATK_EPS/ATK_ITERS)

        whitebox_correct = 0.
        blackbox_correct = 0.
        running_total = 0.
        for batch_idx, (data, labels) in enumerate(test_loader):
            data, labels = data.to(device), labels.to(device)
            # TODO: Perform adversarial attack here
            adv_data = lab2_bcd_return_adv_data(model=whitebox, device=device, dat=data, lbl=labels, eps=ATK_EPS, 
                                                alpha=ATK_ALPHA, iters=ATK_ITERS, rand_start=True, question_label=q_label)
            # Sanity checking if adversarial example is "legal"
            assert(torch.max(torch.abs(adv_data-data)) <= (ATK_EPS + 1e-5)), \
                "torch.max(torch.abs(adv_data-data)) = %g, %s, ATK_EPS=%g" % (torch.max(torch.abs(adv_data-data)), q_label, ATK_EPS)
            assert(adv_data.max() == 1.), "adv_data.max() = %g, %s, ATK_EPS=%g" % (adv_data.max(), q_label, ATK_EPS)
            assert(adv_data.min() == 0.), "adv_data.min() = %g, %s, ATK_EPS=%g" % (adv_data.min(), q_label, ATK_EPS)
            
            # Compute accuracy on perturbed data
            with torch.no_grad():
                # Stat keeping - whitebox
                whitebox_outputs = whitebox(adv_data)
                _,whitebox_preds = whitebox_outputs.max(1)
                whitebox_correct += whitebox_preds.eq(labels).sum().item()
                # Stat keeping - blackbox
                blackbox_outputs = blackbox(adv_data)
                _,blackbox_preds = blackbox_outputs.max(1)
                blackbox_correct += blackbox_preds.eq(labels).sum().item()
                running_total += labels.size(0)
            
            # # Plot some samples
            # if batch_idx == 1:
            #     plt.figure(figsize=(15,5))
            #     for jj in range(12):
            #         plt.subplot(2,6,jj+1);plt.imshow(adv_data[jj,0].cpu().numpy(),cmap='gray');plt.axis("off")
            #     plt.tight_layout()
            #     plt.show()

        # Print final 
        whitebox_acc = whitebox_correct/running_total
        blackbox_acc = blackbox_correct/running_total

        white_acc_dict[q_label].append(whitebox_acc)
        black_acc_dict[q_label].append(blackbox_acc)
        print("Attack Epsilon: {}; Whitebox Accuracy: {}; Blackbox Accuracy: {}".format(ATK_EPS, whitebox_acc, blackbox_acc))
print("Done!")


In [ ]:
figure, axis = plot_lab2_white_black(epsilon_arr=EPS_list_lab2, white_acc_d=white_acc_dict, black_acc_d=black_acc_dict, 
                             label_names=lab2_label)
plt.show()

## Test Robust Models

Don't forget to plot accuracy vs. epsilon curves!

In [ ]:
whitebox = models.NetA()
whitebox.load_state_dict(torch.load()) # TODO: Load your robust models
whitebox = whitebox.to(device)
whitebox.eval(); 

test_acc,_ = test_model(whitebox,test_loader,device)
print("Initial Accuracy of Whitebox Model: ",test_acc)

## Test the model against an adversarial attack

# TODO: Set attack parameters here
#ATK_EPS = ?
#ATK_ITERS = ?
#ATK_ALPHA = ?

whitebox_correct = 0.
running_total = 0.
for batch_idx,(data,labels) in enumerate(test_loader):
    data = data.to(device) 
    labels = labels.to(device)

    # TODO: Perform adversarial attack here
    
    # Sanity checking if adversarial example is "legal"
    assert(torch.max(torch.abs(adv_data-data)) <= (ATK_EPS + 1e-5) )
    assert(adv_data.max() == 1.)
    assert(adv_data.min() == 0.)
    
    # Compute accuracy on perturbed data
    with torch.no_grad():
        whitebox_outputs = whitebox(adv_data)
        _,whitebox_preds = whitebox_outputs.max(1)
        whitebox_correct += whitebox_preds.eq(labels).sum().item()
        running_total += labels.size(0)
    
    # Plot some samples
    if batch_idx == 1:
        plt.figure(figsize=(15,5))
        for jj in range(12):
            plt.subplot(2,6,jj+1);plt.imshow(adv_data[jj,0].cpu().numpy(),cmap='gray');plt.axis("off")
        plt.tight_layout()
        plt.show()

# Print final 
whitebox_acc = whitebox_correct/running_total
print("Attack Epsilon: {}; Whitebox Accuracy: {}".format(ATK_EPS, whitebox_acc))

print("Done!")